In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 34.5 MB/s 
     |████████████████████████████████| 9.3 MB 12.2 MB/s 
     |████████████████████████████████| 108 kB 65.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df_5percent=df[0:2605] 


In [10]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,208,5,1,1,1,...,1,0,5,1,6.023590,1.154529,385024,7.731795,0,1
1,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.567965,5.031528,5361664,6.725206,0,0
2,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.679174,0.000000,188416,5.481215,0,1
3,144,3,4,65535,184,128,5,1,1,1,...,1,0,5,0,1.970041,7.827413,475136,7.771040,0,1
4,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,0,0.000000,0.000000,3724936,7.998396,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,144,3,4,65535,184,224,7,1,1,1,...,1,3,4,0,6.157931,1.014414,25600,5.375449,0,1
2601,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,0.686928,0.003282,29400,2.851589,1,0
2602,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,1,6.336892,2.693930,49152,5.262922,1,0
2603,144,3,4,65535,184,224,3,1,1,1,...,1,2,1,1,0.000000,0.000000,152656,7.972652,0,1


In [11]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1236, 69)
class 1: (1369, 69)


In [12]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [13]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     208.0               5.0   
1      144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     184.0               3.0   
3      144.0   3.0        4.0     65535.0  184.0     128.0               5.0   
4       80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
...      ...   ...        ...         ...    ...       ...               ...   
2600   144.0   3.0        4.0     65535.0  184.0     224.0               7.0   
2601   144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
2602   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
2603   144.0   3.0        4.0     65535.0  184.0     224.0               3.0   
2604   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   

      C

In [14]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2605.0,...,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2605.000000,2.605000e+03,2605.000000,2605.000000,2605.000000
mean,159.483685,10.654894,8.907869,65203.917466,215.639923,222.931286,4.662572,0.988484,0.353551,1.0,...,0.998464,1.388100,3.274472,0.158925,4.878265,2.530080,7.618964e+05,6.400135,0.550096,0.525528
std,800.572286,392.787208,251.709139,4538.700504,1325.811577,49.858173,2.009234,0.106715,0.478164,0.0,...,0.039163,1.703957,1.176895,0.365676,2.595674,2.695024,5.378765e+06,1.118615,0.497580,0.499444
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.948043,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.782408,0.000000,6.451200e+04,5.700895,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.164772,1.521239,1.228800e+05,6.407472,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.517037,4.687140,3.072000e+05,7.380787,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,600.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999956,1.000000,1.000000


In [15]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [16]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [17]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [18]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      51.875
1.0      48.125
dtype: float64

In [19]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
0.0      50.0
1.0      50.0
dtype: float64

In [20]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [21]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [22]:
circuit(weights_init, data[0])

tensor(0.03860724, requires_grad=True)

In [23]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [24]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [25]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [26]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [27]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.08113934, requires_grad=False), tensor(-0.0926955, requires_grad=False)], Y =  1
X = [tensor(-0.07753053, requires_grad=False), tensor(0.02419864, requires_grad=False)], Y = -1
X = [tensor(-0.08128679, requires_grad=False), tensor(-0.08584457, requires_grad=False)], Y =  1
X = [tensor(-0.08008109, requires_grad=False), tensor(-0.05826414, requires_grad=False)], Y = -1
X = [tensor(-0.07597539, requires_grad=False), tensor(0.11595648, requires_grad=False)], Y = -1


In [28]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [29]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0907728 | Accuracy: 0.5187500 
Iter:     2 | Cost: 1.0238674 | Accuracy: 0.5187500 
Iter:     3 | Cost: 1.0184310 | Accuracy: 0.4812500 
Iter:     4 | Cost: 1.0182272 | Accuracy: 0.4812500 
Iter:     5 | Cost: 1.0005883 | Accuracy: 0.5187500 
Iter:     6 | Cost: 1.0473596 | Accuracy: 0.5187500 
Iter:     7 | Cost: 1.0521461 | Accuracy: 0.5187500 
Iter:     8 | Cost: 1.0015229 | Accuracy: 0.5187500 
Iter:     9 | Cost: 1.0092111 | Accuracy: 0.5187500 
Iter:    10 | Cost: 1.0089876 | Accuracy: 0.5187500 
Iter:    11 | Cost: 1.0014222 | Accuracy: 0.5187500 
Iter:    12 | Cost: 0.9999141 | Accuracy: 0.5187500 
Iter:    13 | Cost: 1.0109849 | Accuracy: 0.5187500 
Iter:    14 | Cost: 1.0421901 | Accuracy: 0.5187500 
Iter:    15 | Cost: 1.0689664 | Accuracy: 0.5187500 
Iter:    16 | Cost: 1.0638755 | Accuracy: 0.5187500 
Iter:    17 | Cost: 1.0198263 | Accuracy: 0.5187500 
Iter:    18 | Cost: 1.0038453 | Accuracy: 0.5187500 
Iter:    19 | Cost: 0.9988839 | Accur

In [30]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [31]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.1529454839932503, Accuracy: 45.0%


In [32]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,-1.0
5,1.0,1.0
6,1.0,-1.0
7,1.0,-1.0
8,1.0,-1.0
9,1.0,-1.0


In [33]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         2
         1.0       0.90      0.47      0.62        38

    accuracy                           0.45        40
   macro avg       0.45      0.24      0.31        40
weighted avg       0.86      0.45      0.59        40

0.9
0.47368421052631576
0.6206896551724138
0.23684210526315788
